This notebook is designed to run in a IBM Watson Studio Apache Spark runtime. In case you are running it in an IBM Watson Studio standard runtime or outside Watson Studio, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In [ ]:
!pip install --upgrade pip

In [ ]:
if not ('sc' in locals() or 'sc' in globals()):
    print('It seems you are note running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production')
    
    from pip import main
    main(['install', 'pyspark==2.4.5'])
    
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession

    sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    
    spark = SparkSession \
        .builder \
        .getOrCreate()

# Exercic 2
## Part 1
Now let's calcuate covariance and correlation by ourselfes using ApacheSpark

1st we crate two random RDD's which shouldn't correlate at all.

In [ ]:
import random
rddX = sc.parallelize(random.sample(range(100),100))
rddY = sc.parallelize(random.sample(range(100),100))

Now we calculate the mean

In [ ]:
meanX = rddX.sum()/float(rddX.count())
meanY = rddY.sum()/float(rddY.count())
print(meanX)
print(meanY)

Now we calculate the covariance

In [ ]:
rddXY = rddX.zip(rddY)
covXY = rddXY.map(lambda xy : (float(xy[0])-meanX)*(float(xy[1])-meanY)).sum()/rddXY.count()
covXY

Covariance is not a normalized measure. Therefore we use it to calculate correlation. But before that we need to calculate the indivicual standard deviations first

In [ ]:
from math import sqrt
n = rddXY.count()
sdX = sqrt(rddX.map(lambda x : pow(x-meanX,2)).sum()/n)
sdY = sqrt(rddY.map(lambda x : pow(x-meanY,2)).sum()/n)
print(sdX)
print(sdY)

Now we calculate the correlation

In [ ]:
corrXY = covXY / (sdX * sdY)
corrXY

## Part 2
No we want to create a correlation matrix out of the four RDDs used in the lecture

In [ ]:
from pyspark.mllib.stat import Statistics
import random
column1 = sc.parallelize(range(100))
column2 = sc.parallelize(range(100,200))
column3 = sc.parallelize(list(reversed(range(100))))
column4 = sc.parallelize(random.sample(range(100),100))
data = column1.zip(column2).zip(column3).zip(column4).map(lambda nested : (nested[0][0][0],nested[0][0][1],nested[0][1],nested[1]))
print(Statistics.corr(data))

Congratulations, you are done with Exercice 2